In [1]:
import ee
import geemap
import geopandas as gpd
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1ATx3LY6Hgd3G8-MUKKS5hhxlnOl30rRz60GxiYx3beqIlVhpdjsxonfl60Y

Successfully saved authorization token.


## Cloud value 50 and Satsure time line TCs 

In [ ]:
Map = geemap.Map()
from datetime import date, timedelta,datetime

start_dt = date(2023,10,1)
end_dt = date(2023,10,21)

delta = timedelta(days=10)

dates = []

while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
    if int(start_dt.month)==int(2) and int(start_dt.day)==int(21):
        dates.append(start_dt.isoformat())
        start_dt=date(start_dt.year, 3, 1)
    elif int(start_dt.month)==int(12) and int(start_dt.day)==int(31):
        start_dt=date(start_dt.year+1, 1, 1)
        dates.append(start_dt.isoformat())
        start_dt += delta
  
    elif start_dt.day== 31:
        start_dt=date(start_dt.year, start_dt.month+1, 1)
        dates.append(start_dt.isoformat())
        start_dt += delta

    else:
        dates.append(start_dt.isoformat())
        start_dt += delta  

    
for i in range(len(dates) - 1):
    value = dates[i:i+2]
    delta = timedelta(days=1)
    y,m,d=value[0].split('-')
    intial_date=date(int(y),int(m),int(d))+delta
    print(intial_date,value[1])
#     shp=gpd.read_file("C:/Satsure/GovUP/Acreage/UttarPradeshshps/UttarPradesh/Dist/091034045000000.shp")
    shp=geemap.shp_to_ee("C:/Satsure/SOI_Shapefiles/Odisha/Dist/091026017000000.shp")
#     shapefile = geemap.gdf_to_ee(shp)
    aoi=shp.geometry()
    roi=aoi.transform('EPSG:32645',0.001)

    # Filter Sentinel-2 imagery
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(shp) \
        .filterDate(str(intial_date), value[1]) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 100))

    # Function to calculate NDVI
    def calculate_ndvi(image):
        ndvi = image.normalizedDifference(['B8', 'B4']) \
            .rename('NDVI') \
            .multiply(100).add(100).uint8()  # Scale NDVI values for export

        return image.addBands(ndvi)

    # Apply NDVI calculation function to each image
    ndvi_collection = sentinel2.map(calculate_ndvi)


    # Function to mask NDVI based on blue and green bands
    def mask_ndvi(image):
        blue = image.select('B2')
        green = image.select('B3')
        ndvi = image.select('NDVI')

        # Create mask for blue and green bands
        mask = blue.gt(3600).And(green.gt(3800))

        # Mask NDVI values with 50 where the mask is True
        ndvi_masked = ndvi.where(mask, 50).rename('NDVI_MASKED')

        return image.addBands(ndvi_masked)

    
    
     # Apply masking function to each image
    mask_ndvicol=ndvi_collection.map(mask_ndvi)

    
    color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
           '74A901', '66A000', '529400', '3E8601', '207401', '056201',
           '004C00', '023B01', '012E01', '011D01', '011301']
    pallete = {"min":-1, "max":1, 'palette':color}

    
    # Map the NDVI calculation function over the image collection
    mask_ndvi = mask_ndvicol.select('NDVI_MASKED').max().reproject(crs='EPSG:32645').clip(roi)
#     mask_ndvi = mask_ndvicol.select('NDVI_MASKED').max().clip(shapefile)
    Map.addLayer(mask_ndvi,pallete,f'{value[1]}_ndvi')


    ####give the file name in description and folder name  below
    geemap.ee_export_image_to_drive(mask_ndvi, description=f'{value[1]}_091026017000000_STC',
                                    folder='CR', 
                                    region= roi, scale=10, maxPixels=316672400)

# FCC

In [4]:
AOI = geemap.shp_to_ee('D:/GOVUP/Group-D-District_Boundary_28feb2024/Pratapgarh_utm.shp')
# AOI = geemap.geojson_to_ee('E:/Soham/merged_waldo.geojson')
roi=AOI.geometry()
# shp=gpd.read_file("C:/Satsure/GovUP/91034036000000/shp/91034036000000.shp")
# AOI = geemap.shp_to_ee(shp)
# roi=AOI.geometry()
S2 = ee.ImageCollection("COPERNICUS/S2_SR").select('B2','B3','B4','B8')
# #image1=S2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',30)).filter(ee.Filter.date('2021-11-01', '2021-11-15')).filter(ee.Filter.bounds(AOI)).select('B2','B3','B4','B8').map(addNDVI).max().clip(AOI)
collection1=S2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',100)).filter(ee.Filter.date('2024-05-20', '2024-05-30')).filter(ee.Filter.bounds(AOI)).mosaic().clip(AOI)
vizParams = {
  'bands': ['B8', 'B3', 'B2'],
  'min': 0,
  'max': 4000,
  'gamma': [0.95, 1.1, 1]
}
Map = geemap.Map()
Map.addLayer(collection1, vizParams, 'False color composite')
Map.addLayerControl()
Map.centerObject(AOI, zoom = 8)
Map

Map(center=[25.87261897761894, 81.83000411490946], controls=(WidgetControl(options=['position', 'transparent_b…

In [5]:
geemap.ee_export_image_to_drive(collection1, description='FCC_2024May30_Pratapgarh', folder='earthengine', region= roi, scale=10, maxPixels=392036832)